# Data Cleaning <a class="tocSkip">

In [ ]:
# Loading useful extensions
%load_ext autoreload
%autoreload
%load_ext nb_black
%matplotlib inline

In [ ]:
# Import packages
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

# Load Data

In [ ]:
df = pd.read_csv("sales_data_2015.csv")

# Filtering Data

In [ ]:
def general_data_cleaning(df):
    """
    Cleaning the data set by deleting unused columns and filtering columns for unrealistic data
    """
    # Deleting 0 values from the data set
    df = df[(df.yr_built != 0) & (df.tot_sqft != 0) & (df.price != 0)]
    # Deleting columns that are mostly NaN values and unused columns
    df = df.copy().drop(
        ["easmnt", "apt", "Unnamed: 0", "usable", "zip", "block", "lot"], axis=1
    )
    # Drop duplicates
    df = df.drop_duplicates(df.columns, keep="last")
    # Drop nan values and reseting the index
    df = df.dropna()
    df = df.reset_index(drop=True)
    # Remove observations that fall outside those caps
    df = df[(df["price"] > 100000) & (df["price"] < 5000000)]
    df = df[(df["tot_unit"] > 0) & (df["tot_unit"] != 2261)]
    return df

# Cleaning Data for Regression Analysis

In [ ]:
def one_hot_encoding(df):
    """
    One hot encoding all the categorical features
    """
    df_categorical = df[["borough", "bldg_ctgy", "tax_cls_s", "tax_cls_p"]]
    # Changing the data type
    df_categorical["borough"] = df_categorical["borough"].astype(object)
    df_categorical["tax_cls_s"] = df_categorical["tax_cls_s"].astype(object)
    # Convert categorical variables into dummy/indicator variables (i.e. one-hot encoding).
    one_hot_encoded = pd.get_dummies(df_categorical)
    return one_hot_encoded

In [ ]:
def scaling_nummerical_features(df):
    """
    Scaling all the nummerical columns
    """
    # Selecting all the nummerical features
    df_nummerical = df[
        [
            "res_unit",
            "com_unit",
            "tot_unit",
            "land_sqft",
            "tot_sqft",
            "yr_built",
            "price",
        ]
    ]
    # Transform the numeric features using log(x + 1)
    skewed = df_nummerical[df_nummerical.columns].apply(
        lambda x: skew(x.dropna().astype(float))
    )
    skewed = skewed[skewed > 0.75]
    skewed = skewed.index
    df_nummerical[skewed] = np.log1p(df_nummerical[skewed])
    # Scale the features
    scaler = StandardScaler()
    scaler.fit(df_nummerical[df_nummerical.columns])
    scaled = scaler.transform(df_nummerical[df_nummerical.columns])

    for i, col in enumerate(df_nummerical.columns):
        df_nummerical[col] = scaled[:, i]

    return df_nummerical

In [ ]:
def regression_data_cleaning(df):
    """
    Cleaning data for regression models by removing unusable columns 
    Scaling nummerical columns
    One hot encoding categorical columns
    """
    # General data cleaning first
    df = general_data_cleaning(df)
    # Extracting Sale_id
    df_sale = df[["Sale_id"]]
    # Removing unused columns
    df = df.copy().drop(
        [
            "bbl_id",
            "address",
            "sale_date",
            "long",
            "lat",
            "year",
            "bldg_cls_p",
            "bldg_cls_s",
        ],
        axis=1,
    )
    # Run scaling and one hot encoding
    df = pd.concat([scaling_nummerical_features(df), one_hot_encoding(df)], axis=1)
    # Adding Sale_id back to the data frame
    df = pd.concat([df_sale, df], axis=1)
    return df

# Output

In [ ]:
df_reg = regression_data_cleaning(df)
df_eda = general_data_cleaning(df)

In [ ]:
df_reg.to_csv("df_reg.csv")
df_eda.to_csv("df_eda.csv")